In [26]:
"""
TESTING
"""

import random

def normalize(probabilities):
    total_in = sum(probabilities.values())
    for base, value in probabilities.items():
        probabilities[base] = value / total_in
    return probabilities
        
print("Test normalize: ", normalize({'A': 0.1, 'C': 0.1, 'G': 0.1, 'T': 0.1}))
print()

def weighted_die(probabilities):
    rand_p = random.uniform(0, 1)
    for t, p in probabilities.items():
        rand_p = rand_p - p
        if rand_p <= 0:
            return t



def testProbabilities(Probabilities):
    dict = {'AA' : 0, 'TT' : 0, 'CC' : 0, 'GG' : 0, 'AT' : 0}
    for i in range(10000):
        increment = weighted_die(Probabilities)
        dict[increment] += 1
    return dict


print("Test weighted die: ", weighted_die({'AA' : 0.2,
                                           'TT' : 0.2,
                                           'CC' : 0.1,
                                           'GG' : 0.1,
                                           'AT' : 0.4}))
print()
  
print(testProbabilities({'AA' : 0.2,
                                           'TT' : 0.2,
                                           'CC' : 0.1,
                                           'GG' : 0.1,
                                           'AT' : 0.4}))

# around 2000 AA, 2000 TT, 1000 CC, 1000 GG, and 4000 AT


Test normalize:  {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}

Test weighted die:  AT

{'AA': 2065, 'TT': 1946, 'CC': 1027, 'GG': 972, 'AT': 3990}


In [29]:
import random

def profile_generated_string(text, profile, k):
    n = len(text)
    probabilities = {}
    for i in range(0,n-k+1):
        probabilities[text[i:i+k]] = prb(text[i:i+k], profile)
    probabilities = normalize(probabilities)
    return weighted_die(probabilities)


def prb(pattern, profile):
    p = 1
    for i in range(len(pattern)):
        p *= profile[pattern[i]][i]
    return p

def normalize(probabilities):
    total_in = sum(probabilities.values())
    for base, value in probabilities.items():
        probabilities[base] = value / total_in
    return probabilities
        
def weighted_die(probabilities):
    rand_p = random.uniform(0, 1)
    for t, p in probabilities.items():
        rand_p = rand_p - p
        if rand_p <= 0:
            return t
        
print(profile_generated_string("AAACCCAAACCC", 
                               {'A': [0.5, 0.1], 'C': [0.3, 0.2], 'G': [0.2, 0.4], 'T': [0.0, 0.3]}, 
                               2))



CA


In [2]:
"""
PSEUDOCODE

    GibbsSampler(Dna, k, t, N)
        randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
        BestMotifs ← Motifs
        for j ← 1 to N
            i ← randomly generated integer between 1 and t
            Profile ← profile matrix formed from all strings in Motifs except for Motifi
            Motifi ← Profile-randomly generated k-mer in the i-th string
            if Score(Motifs) < Score(BestMotifs)
                BestMotifs ← Motifs
        return BestMotifs
        
"""
import random

def gibbs_sampler(dna, k, t, N):
    best_mtfs = randomized_motif_search(dna, k, t)
    for _ in range(N):
        current_mtfs = randomized_motif_search(dna, k, t)
        i = random.randint(1, t)
        current_mtfs[i-1] = random_motifs(dna, k, t)[i-1]
        if score(current_mtfs) > score(best_mtfs):
            best_mtfs = current_mtfs
    return best_mtfs
    
    
def randomized_motif_search(dna, k, t):
    ms = random_motifs(dna, k, t)
    best_motifs = ms
    while True:
        profile = profile_pseudo(ms)
        ms = motifs(profile, dna)
        if score(ms) < score(best_motifs):
            best_motifs = ms
        else:
            return best_motifs        
      
        

def random_motifs(dna, k, t):
    t_dna = len(dna)
    mx_k = len(dna[0])
    random_motifs = []
    for i in range(t_dna):
        k_rand_j = random.randint(1, mx_k-k)
        random_motifs.append(dna[i][k_rand_j:k_rand_j+k])
    return random_motifs


def profile_pseudo(motifs):
    t = len(motifs)
    k = len(motifs[0])
    n_b = 4
    profile = count_pseudo(motifs)
    for base, l in profile.items():
        nl = [c / (t+n_b) for c in l]
        profile[base] = nl
    return profile


def motifs(profile, dna):
    n_strings = len(dna)
    k = len(profile["A"])
    motifs = []
    for s in range(n_strings):
        motifs.append(profile_kmer(dna[s], k, profile))        
    return motifs

def profile_kmer(text, k, profile):
    n = len(text)
    mx_p = -1
    result = ""
    for i in range(n-k+1):
        pattern = text[i:i+k]
        if prb(pattern, profile) > mx_p:
            mx_p = prb(pattern, profile)
            result = pattern
    return result

def prb(pattern, profile):
    p = 1
    for i in range(len(pattern)):
        p *= profile[pattern[i]][i]
    return p


def score(motifs):
    count = count_pseudo(motifs)
    base_consensus = consensus(motifs)
    score = 0
    i = 0
    for b in base_consensus:
        for k, l in count.items():
            if b != k:
                score += l[i] 
        i += 1
    return score

def count_pseudo(motifs):
    count = {} 
    t = len(motifs)
    k = len(motifs[0])
    for symbol in "ACGT":
        count[symbol] = [1] * k
    for i in range(t):
        for j in range(k):
            symbol = motifs[i][j]
            count[symbol][j] += 1
    return count

def consensus(motifs):
    k = len(motifs[0])
    cnt = count(motifs)
    result = ""
    for b in range(k):
        mx = 0
        frequent_symbol = ""
        for symbol in "ACGT":
            if cnt[symbol][b] > mx:
                mx = cnt[symbol][b]
                frequent_symbol = symbol
        result += frequent_symbol
    return result

def count(motif):
    count = {} 
    t = len(motif)
    k = len(motif[0])
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
             count[symbol].append(0)
    for i in range(t):
        for j in range(k):
            symbol = motif[i][j]
            count[symbol][j] += 1
    return count


dna = ["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA", 
       "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG", 
       "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
       "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC", 
       "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]
t = 5 
k = 8
N = 100

gibbs_test_motifs = gibbs_sampler(dna, k, t, N)

print("Test gibbs motif search 100 iterations: ", gibbs_test_motifs)
print()
print("Score: ", score(gibbs_test_motifs))

Test gibbs motif search 100 iterations:  ['GTAAACGG', 'AGGTGCCA', 'CCGAAAGA', 'GTCGGTGA', 'GCAATGTT']

Score:  44


In [5]:
DosR_dna = ["GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC",
            "CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG",
            "ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC",
            "GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC",
            "GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG",
            "CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA",
            "GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA",
            "GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG",
            "GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG",
            "TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"]
t = len(DosR_dna) 
k = 15
N = 10000

from datetime import datetime
startTime = datetime.now()        

DosR_gibbs_motifs = gibbs_sampler(DosR_dna, k, t, N)

print(datetime.now() - startTime)

print()
print("Gibbs motif search 100,000 iterations for Dos_R: ", DosR_gibbs_motifs)
print()
print("Score: ", score(DosR_gibbs_motifs))

0:03:45.814378

Gibbs motif search 100,000 iterations for Dos_R:  ['GGCTCAGTTAGGGCC', 'GACTCGAGGAGGATC', 'TCGCGCCGCGTCCAC', 'CGGCCAGGGCGCCGG', 'AGGTCAAACCGGGTG', 'CGCCGGCAGCGTGCG', 'GTCCCCAAGTGGATC', 'CACTCAAAGCCTACT', 'CTGGGAGATAGCGGT', 'CTGTCATCGAGCGAT']

Score:  120
